In [ ]:
# Install necessary packages (run once)
!pip install --quiet tensorflow scikeras scikit-learn numpy

# Import libraries
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD, Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Build model function with hyperparameters
def build_model(hidden_units=128, optimizer='adam'):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(hidden_units, activation='relu'),
        Dense(10, activation='softmax')
    ])
    if optimizer == 'sgd':
        opt = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    else:
        opt = Adam()
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Manually setting hyperparameters and training
print("----- Manual Hyperparameter Setting -----")
manual_model = build_model(hidden_units=64, optimizer='sgd')
manual_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1, verbose=2)
loss, accuracy = manual_model.evaluate(x_test, y_test, verbose=0)
print(f"Manual Setting Test Accuracy: {accuracy:.4f}")

# Grid Search for Hyperparameter Tuning
print("\n----- Grid Search Hyperparameter Tuning -----")

clf = KerasClassifier(model=build_model, verbose=0)

param_grid = {

    'model__hidden_units': [64, 128],
    'model__optimizer': ['adam', 'sgd'],
    'batch_size': [64, 128],
    'epochs': [5]
}

grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)

print("\nBest Hyperparameters from Grid Search:")
print(grid_result.best_params_)

best_model = grid_result.best_estimator_.model_
test_loss, test_acc = best_model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy with Best Hyperparameters: {test_acc:.4f}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
----- Manual Hyperparameter Setting -----


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/5
844/844 - 3s - 3ms/step - accuracy: 0.8809 - loss: 0.4207 - val_accuracy: 0.9423 - val_loss: 0.2087
Epoch 2/5
844/844 - 3s - 3ms/step - accuracy: 0.9355 - loss: 0.2253 - val_accuracy: 0.9577 - val_loss: 0.1628
Epoch 3/5
844/844 - 4s - 5ms/step - accuracy: 0.9488 - loss: 0.1761 - val_accuracy: 0.9645 - val_loss: 0.1375
Epoch 4/5
844/844 - 2s - 2ms/step - accuracy: 0.9584 - loss: 0.1458 - val_accuracy: 0.9663 - val_loss: 0.1215
Epoch 5/5
844/844 - 2s - 2ms/step - accuracy: 0.9639 - loss: 0.1253 - val_accuracy: 0.9692 - val_loss: 0.1155
Manual Setting Test Accuracy: 0.9612

----- Grid Search Hyperparameter Tuning -----


AttributeError: 'super' object has no attribute '__sklearn_tags__'